In [4]:
"""
PROYECTO 1 | Evaluacion de planeacion de objetos.

Autor: Magno Gomez Ricardo Leon

Fecha de entrega: 24 de abril del 2024
"""

# Bibliotecas
!pip install criticalpath

import numpy as np
import pandas as pd
from criticalpath import Node
from google.colab import drive

drive.mount("/content/gdrive")

# Variables

excel = input("De ubicacion del Excel: ",) # El usuario da el archivo.
archivo = pd.read_excel(excel) # Asignamos un nombre al archivo generado en py.

actividades = list(archivo["Actividad"])
descripcion = list(archivo["Descripcion"])
columna_precedentes = list(archivo["Precedentes"])
duracion = list(archivo["Duracion"])

# Clases

class Grafica_dirigida:
  """Clase de Grafica dirigida."""

  def __init__(self, nodos, arcos):
    """Constructor encargado de recibir los atributos de un objeto y validar que
    mismos cumplan las condiciones para pertenecer a la clase."""
    if type(nodos) == list and type(arcos) == list: # verificamos listas
      Bool1 = True
      Bool2 = True

      for n in nodos:
        if type(n) != int: # los nodos deben ser enteros
          Bool1 = False

      if Bool1 == False:
        print("Alguno de los nodos no es un entero, verifica tus datos.")

      if Bool1 == True:
        self.nodos = nodos

      for a in arcos:
        if type(a) != tuple or len(a) != 2: # los arcos deben ser tuplas
          Bool2 = False

        if a[0] not in nodos or a[1] not in nodos:
          Bool2 = False

      if Bool2 == False:
        print("Hay un error en los arcos, verifica tus datos.")

      if Bool2 == True:
        self.arcos = arcos

  def antecendetes(self, nodo):
    """Metodo para revisar los antecedentes de un nodo especifico."""

    if nodo not in self.nodos:
      print(nodo, " no pertenece al conjunto de nodos")
      return []

    else:
      ant = []
      for x in self.arcos:
        if nodo == x[-1]:
          ant.append(x[0])

      return ant

  def sucesores(self, nodo):
    """Metodo para revisar los sucesores de un nodo especifico."""

    if nodo not in self.nodos:
        print(nodo, " no pertenece al conjunto de nodos")
        return []

    else:
      ant = []
      for x in self.arcos:
        if nodo == x[0]:
          ant.append(x[-1])

      return ant

# Funciones

def dar_lista_precedentes(columna):
  """Funcion dedicada a crear una lista de los Predecesores de una actividad."""
  lista_precedentes = []
  for elemento in columna:

      if isinstance(elemento, (int, float)):

        if type(elemento) == float:
          # Elimino el Nan, lo hago 0.
          elemento = 0
          listita = []
          lista_precedentes.append(listita)

        else:
          #Para cualquier otro entero lo agrego a la lista.
          listita = []
          listita.append(elemento)
          lista_precedentes.append(listita)

      elif isinstance(elemento, str): # Si es cadena...
          # Divido la cadena en elementos separados y se hacen enteros.
          elemento = elemento.replace(" ", "") # Elimino espacios.

          # Se hace una lista en la que se guardan nuestros nuevos enteros.
          elementos = [int(valor) for valor in
                       elemento.split(",") if valor.isdigit()]

          # Agrego la nueva lista dentro de la principal
          lista_precedentes.append(elementos)

  return lista_precedentes

precedentes = dar_lista_precedentes(columna_precedentes) # Hacemos lista de precedentes.

def armar_nodos(archivo):
  """Da una lista de nodos según el archivo seleccionado"""
  nodos = []
  nodos = [] + list(archivo["Actividad"]) # Los nodos de la actividad
  return nodos

nodos = armar_nodos(archivo)

def armar_arcos(archivo):
  """Da una lista de arcos según el archivo seleccionado"""
  arcos = []
  for i in range(len(archivo)):
    if type(precedentes[i]) == int:
      arcos.append((precedentes[i], archivo.iloc[i,0]))

    elif type(precedentes[i]) == list:
      for j in precedentes[i]:
        arcos.append((j, archivo.iloc[i,0]))
  return arcos

arcos = armar_arcos(archivo)

def informacion_proyecto(nodos, precedentes, duracion):
  """Proporciona la duración del proyecto y las actividades criticas."""
  datos = Node("Proyecto")
  n_nodos = {}

# Longitud de Duracion es igual a longitud de Actividades. SPDG.
  for i in range(len(duracion)):
    n_nodos[nodos[i]] = datos.add(Node(str(nodos[i]), duration = duracion[i]))

  for j in range(len(duracion)):
      for precedente_i in precedentes[j]:
          if precedente_i in n_nodos:
            datos.link(n_nodos[precedente_i], n_nodos[nodos[j]])

  datos.update_all()

  return datos

# CUERPO MAIN

def proyecto_1():
  """Funcion encargada de ejecutar el proyecto"""

  precedentes = dar_lista_precedentes(columna_precedentes)
  nodos = armar_nodos(archivo)
  arcos = armar_arcos(archivo)

  G = Grafica_dirigida(nodos, arcos)

  proyecto = informacion_proyecto(nodos, precedentes, duracion)
  duracion_proyecto = proyecto.duration
  n_actividades_criticas = proyecto.get_critical_path()

  print("------------------------------------------------------------------")

  with open("Reporte_Proyecto_1.txt", "w") as doc:
    doc.write("El tiempo necesario para la elaboración del proyecto es de ")
    doc.write(str(duracion_proyecto))
    doc.write(" semanas.\n")
    doc.write("Las actividades criticas son:\n")
    doc.write("\n")

    lista_v = []
    for i in range(len(n_actividades_criticas)):
      lista_v.append(n_actividades_criticas[i].name)

    for num in lista_v:
      n = descripcion[int(num)-1]
      doc.write(f"{num} - {n}\n")

    # Esto es solo para mostrar el contenido del archivo después de escribirlo
  with open("Reporte_Proyecto_1.txt") as doc:
    print(doc.read())

if __name__ == "__main__":
  proyecto_1()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
De ubicacion del Excel: /content/gdrive/MyDrive/Programación/Proyectos/Creacion_de_la_carrera_de_mat_ap.xlsx
------------------------------------------------------------------
El tiempo necesario para la elaboración del proyecto es de 124 semanas.
Las actividades criticas son:

1 - Decision preliminar
2 - Determinacion y analisis de objetivos
3 - Sugerencia del plan de estudios
4 - Presentacion de la propuesta
6 - Consulta con el sector academico de la UNAM
11 - Analisis, discusion e incorporacion de sugerencias de la UNAM
14 - Analisis comparativo de planes similares en la UNAM
15 - Adicion de consideraciones nacionales
18 - Presentacion en el consejo tecnico de la Facultad de Ciencias
16 - Adicion de consideraciones internacionales
17 - Elaboracion del documento administrativo
20 - Elaboracion del plan de estudios final
21 - Presentacion de propuesta def